In [1]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import pytz


# Start the timer
start_time = time.time()

base_url = 'https://www.subito.it/annunci-friuli-venezia-giulia/vendita/appartamenti/trieste/?o={}'

def get_individual(url):
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')

    title = soup.find('div', class_='general-info_ad-info___SSdI').find('h1').text if soup.find('div', class_='general-info_ad-info___SSdI') else None

    price_tag = soup.find('p', class_='index-module_price__N7M2x AdInfo_ad-info__price__tGg9h index-module_large__SUacX')
    price = price_tag.text.strip() if price_tag else None

    area_div = soup.find('div', class_='AdInfo_ad-info__location__nd8JQ')

    # Extract the text from the span tag inside the div
    area = area_div.find('span', class_='index-module_sbt-text-atom__ed5J9').text.strip() if area_div else None


    descrizione = soup.find('section', class_='grid_detail-component__7sBtj grid_description__rEv3i').get_text(strip=True)

    other_tag = soup.find('div', id='feature-list-section_detail-chip-container__by96k')
    p_tags = other_tag.find_all('p', class_='index-module_sbt-text-atom__ed5J9 index-module_token-caption__TaQWv size-normal index-module_weight-book__WdOfA StaticChip-module_static-chip__va4RV StaticChip-module_medium__OZRaA') if other_tag else []

    # Extract the contents of the p tags and store them in a list
    others = [p.get_text(strip=True) for p in p_tags]
    others = ', '.join(others)

    agenzia_tag = soup.find('p', class_='index-module_sbt-text-atom__ed5J9 index-module_token-subheading__SH1NS size-normal index-module_weight-semibold__MWtJJ UserDetails_user-name__sjn9j')
    agenzia = agenzia_tag.text.strip() if agenzia_tag else None

    #Function to clean the values from Locali, Bagni, and Piano, before adding to the dataframe
    def clean_piano_value(value):
        # Check if the value is '-'
        if value.strip() == '-':
            return 0

        # Check if the value is numeric or contains a standalone number
        if any(char.isdigit() for char in value.split()):
            # Extract the numeric part from the value
            numeric_part = ''.join(filter(str.isdigit, value))
            return int(numeric_part)
        else:
            # If the value is a string without a standalone number, return 1
            return 1

    #For this particular function, I had to extract the number from the string, if it contains a number, this is because Bagni
    #and Locali values has there own strings to be unique,e.g they have 'Piu di 3', which means more than 3, so in this case, it would be wise to get the exact number, not just allocating it to 1 unlike the case of Piano

    def clean_bagni_and_locali_value(value):
        # Check if the value is '-'
        if value.strip() == '-':
            return 0

        # Check if the value is numeric or contains a number
        if any(char.isdigit() for char in value):
            # Extract the numeric part from the value
            numeric_part = ''.join(filter(str.isdigit, value))
            return int(numeric_part)
        else:
            # Return 1 if the value is a string
            return 1








    # Initialize dictionaries to store the results
    DB_features = {}


    # Find the ul tag with class "feature-list_feature-list__RDCLn"
    ul_tag = soup.find('ul', class_='feature-list_feature-list__RDCLn')
    desired_labels = ['Locali', 'Superficie', 'Bagni','Piano', 'Riscaldamento', 'Classe energetica','Stato','Parcheggio']

    # Loop through the li elements and extract all labels and values
    lis = soup.find_all('li', class_='feature-list_feature__8a4rn')
    for li in lis:
        label = li.find('span', class_='feature-list_label__Jf58a').text
        value = li.find('span', class_='feature-list_value__pgiul').text
        if label in desired_labels:
            label = "MQ" if label == "Superficie" else label
            value = clean_piano_value(value) if label == "Piano" else value
            value = clean_bagni_and_locali_value(value) if label == "Bagni" else value
            value = clean_bagni_and_locali_value(value) if label == "Locali" else value
            DB_features[label] = value


    # Include the date and time the property was extracted
    extracted_datetime = datetime.now()

    # Get the GMT time zone
    gmt = pytz.timezone('GMT')

    # Convert the extracted_datetime to GMT time zone
    extracted_datetime_gmt = extracted_datetime.astimezone(gmt)





    house_details = {
        'Date_GMT': extracted_datetime_gmt.strftime('%Y-%m-%d %H:%M:%S %Z'),
        'Titolo': title,
        'Area': area,
        'Link': url,
        'Prezzo': price,
        'Dettagli': descrizione,
        'Other Characteristics': others,
        'Agenzia': agenzia
    }

    house_details.update(DB_features)

    return house_details

def scrape_data():
    url = base_url.format(1)
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')

    total_pages = (int(soup.find('div', class_='listing-heading ListingHeading_listing-heading__QSQ4k').find('p').text.split()[0]) + 32) // 33



    all_urls = []
    for page_number in range(1, total_pages + 1):
        url = base_url.format(page_number)
        req = requests.get(url).text
        soup = BeautifulSoup(req, 'html.parser')
        house_list = soup.find_all('div', class_='items__item item-card item-card--big BigCard-module_card__Exzqv')

        for house in house_list:
            house_url = house.find('a')['href']
            all_urls.append(house_url)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_individual, url) for url in all_urls]

    details_list = [future.result() for future in futures]
    df = pd.DataFrame(details_list)

    df['MQ'] = df['MQ'].str.replace('mq', '').str.strip()
    df['MQ'] = pd.to_numeric(df['MQ'],errors = 'coerce')

    df['Prezzo'] = df['Prezzo'].str.replace('€', '').replace(".", "").str.strip()
    df['Prezzo'] = pd.to_numeric(df['Prezzo'],errors = 'coerce')

    df['Prezzo_al_mq'] = df['Prezzo']/df['MQ']

    df.insert(5, 'Prezzo_al_mq', df.pop('Prezzo_al_mq'))
    df.insert(6, 'Locali', df.pop('Locali'))
    df.insert(7, 'MQ', df.pop('MQ'))
    df.insert(8, 'Bagni', df.pop('Bagni'))
    df.insert(9, 'Piano', df.pop('Piano'))


    return df



# Scrape data and print the resulting DataFrame
dataframe = scrape_data()


# Calculate the elapsed time
elapsed_time = time.time() - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")


Elapsed time: 157.23421573638916 seconds


In [2]:
dataframe.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Prezzo_al_mq,Locali,MQ,Bagni,Piano,Dettagli,Other Characteristics,Agenzia,Stato,Parcheggio,Riscaldamento,Classe energetica
0,2023-08-03 15:25:00 GMT,Appartamento a Servola,Trieste (TS),https://www.subito.it/appartamenti/appartament...,195.0,1.560000,4,125,2,2,DescrizioneVendessi appartamento a Servola. L'...,"Balcone, Disponibilità immediata",None,Buono - abitabile,Posto auto riservato,Centralizzato,-
1,2023-08-03 15:25:00 GMT,Bilocale con balcone e cantina,Trieste (TS),https://www.subito.it/appartamenti/bilocale-co...,80.0,1.250000,2,64,1,5,DescrizioneVIA CARPINETO - In contesto silenzi...,"Balcone, Ascensore, Disponibilità immediata, A...",TECNOCASA - SAN GIOVANNI SRL,Buono - abitabile,Posto auto libero,Centralizzato,G
2,2023-08-03 15:25:00 GMT,Appartamentino monolocale Baiamonti bassa,Trieste (TS),https://www.subito.it/appartamenti/appartament...,39.0,1.560000,2,25,1,4,DescrizioneVendo appartamentino tipo monolocal...,"Arredato, Disponibilità immediata",None,Buono - abitabile,-,Altro,In attesa di certificazione
3,2023-08-03 15:25:00 GMT,Appartamento con terrazza,Trieste (TS),https://www.subito.it/appartamenti/appartament...,75.0,1.470588,2,51,1,4,DescrizioneAppartamento con terrazza e box aut...,"Balcone, Ascensore, Disponibilità immediata",AL QUADRATO IMMOBILIARE,Buono - abitabile,-,Centralizzato,In attesa di certificazione
4,2023-08-03 15:25:00 GMT,SAN LUIGI - Ultimo piano con vista mare,Trieste (TS),https://www.subito.it/appartamenti/san-luigi-u...,180.0,2.250000,3,80,1,3,DescrizioneIn zona verde e residenziale appart...,"Ultimo piano, Disponibilità immediata, Aria co...",OMEGA Gruppo Immobiliare,Ottimo - ristrutturato,Box privato,Altro,In attesa di certificazione


In [3]:
dataframe.columns

Index(['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 'Prezzo_al_mq',
       'Locali', 'MQ', 'Bagni', 'Piano', 'Dettagli', 'Other Characteristics',
       'Agenzia', 'Stato', 'Parcheggio', 'Riscaldamento', 'Classe energetica'],
      dtype='object')

In [4]:
GS_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo',
              'Locali', 'MQ', 'Bagni','Piano', 'Dettagli', 'Agenzia']
mainGS_df = dataframe[GS_columns]

#Geting the dataframe for the Database
DB_columns = ['Date_GMT', 'Titolo', 'Area', 'Link', 'Prezzo', 'Prezzo_al_mq',
       'Locali', 'MQ', 'Bagni', 'Piano', 'Dettagli', 'Other Characteristics',
       'Agenzia', 'Stato', 'Parcheggio', 'Riscaldamento', 'Classe energetica']

mainDB_df = dataframe[DB_columns]

In [5]:
mainGS_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Locali,MQ,Bagni,Piano,Dettagli,Agenzia
0,2023-08-03 15:25:00 GMT,Appartamento a Servola,Trieste (TS),https://www.subito.it/appartamenti/appartament...,195.0,4,125,2,2,DescrizioneVendessi appartamento a Servola. L'...,None
1,2023-08-03 15:25:00 GMT,Bilocale con balcone e cantina,Trieste (TS),https://www.subito.it/appartamenti/bilocale-co...,80.0,2,64,1,5,DescrizioneVIA CARPINETO - In contesto silenzi...,TECNOCASA - SAN GIOVANNI SRL
2,2023-08-03 15:25:00 GMT,Appartamentino monolocale Baiamonti bassa,Trieste (TS),https://www.subito.it/appartamenti/appartament...,39.0,2,25,1,4,DescrizioneVendo appartamentino tipo monolocal...,None
3,2023-08-03 15:25:00 GMT,Appartamento con terrazza,Trieste (TS),https://www.subito.it/appartamenti/appartament...,75.0,2,51,1,4,DescrizioneAppartamento con terrazza e box aut...,AL QUADRATO IMMOBILIARE
4,2023-08-03 15:25:00 GMT,SAN LUIGI - Ultimo piano con vista mare,Trieste (TS),https://www.subito.it/appartamenti/san-luigi-u...,180.0,3,80,1,3,DescrizioneIn zona verde e residenziale appart...,OMEGA Gruppo Immobiliare


In [6]:
mainDB_df.head()

,Date_GMT,Titolo,Area,Link,Prezzo,Prezzo_al_mq,Locali,MQ,Bagni,Piano,Dettagli,Other Characteristics,Agenzia,Stato,Parcheggio,Riscaldamento,Classe energetica
0,2023-08-03 15:25:00 GMT,Appartamento a Servola,Trieste (TS),https://www.subito.it/appartamenti/appartament...,195.0,1.560000,4,125,2,2,DescrizioneVendessi appartamento a Servola. L'...,"Balcone, Disponibilità immediata",None,Buono - abitabile,Posto auto riservato,Centralizzato,-
1,2023-08-03 15:25:00 GMT,Bilocale con balcone e cantina,Trieste (TS),https://www.subito.it/appartamenti/bilocale-co...,80.0,1.250000,2,64,1,5,DescrizioneVIA CARPINETO - In contesto silenzi...,"Balcone, Ascensore, Disponibilità immediata, A...",TECNOCASA - SAN GIOVANNI SRL,Buono - abitabile,Posto auto libero,Centralizzato,G
2,2023-08-03 15:25:00 GMT,Appartamentino monolocale Baiamonti bassa,Trieste (TS),https://www.subito.it/appartamenti/appartament...,39.0,1.560000,2,25,1,4,DescrizioneVendo appartamentino tipo monolocal...,"Arredato, Disponibilità immediata",None,Buono - abitabile,-,Altro,In attesa di certificazione
3,2023-08-03 15:25:00 GMT,Appartamento con terrazza,Trieste (TS),https://www.subito.it/appartamenti/appartament...,75.0,1.470588,2,51,1,4,DescrizioneAppartamento con terrazza e box aut...,"Balcone, Ascensore, Disponibilità immediata",AL QUADRATO IMMOBILIARE,Buono - abitabile,-,Centralizzato,In attesa di certificazione
4,2023-08-03 15:25:00 GMT,SAN LUIGI - Ultimo piano con vista mare,Trieste (TS),https://www.subito.it/appartamenti/san-luigi-u...,180.0,2.250000,3,80,1,3,DescrizioneIn zona verde e residenziale appart...,"Ultimo piano, Disponibilità immediata, Aria co...",OMEGA Gruppo Immobiliare,Ottimo - ristrutturato,Box privato,Altro,In attesa di certificazione
